In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movierecommenderdataset/movies.csv
/kaggle/input/movierecommenderdataset/ratings.csv


# build a movie recommendation engine

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df=pd.read_csv('/kaggle/input/movierecommenderdataset/movies.csv')
df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [4]:
df.shape

(9742, 3)

In [5]:
df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


# function for combine the values into a single string

In [9]:
def get_important_features(data):
    important_features=[]
    for i in range(0,data.shape[0]):
        important_features.append(data['title'][i]+' '+data['genres'][i])
    return important_features

In [10]:
df['important_features']=get_important_features(df)
df

,movieId,title,genres,important_features
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995) Adventure|Animation|Children|...
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji (1995) Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995) Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995) Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995) Comedy
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic (2017) Acti...
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,No Game No Life: Zero (2017) Animation|Comedy|...
9739,193585,Flint (2017),Drama,Flint (2017) Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,Bungo Stray Dogs: Dead Apple (2018) Action|Ani...


# convert the text to a matrix

In [11]:
cm=CountVectorizer().fit_transform(df['important_features'])

# get the cosine similarity matrix

In [12]:
cs=cosine_similarity(cm)
print(cs)

[[1.         0.63245553 0.28867513 ... 0.         0.125      0.11785113]
 [0.63245553 1.         0.18257419 ... 0.         0.         0.        ]
 [0.28867513 0.18257419 1.         ... 0.         0.         0.13608276]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.125      0.         0.         ... 0.         1.         0.        ]
 [0.11785113 0.         0.13608276 ... 0.         0.         1.        ]]


In [31]:
title=df['title'][33]
title

'Dead Man Walking (1995)'

In [33]:
movie_id=df[df.title==title]['movieId'].values[0]
movie_id

36

# create a list for similarity score (movie id,similarity score)

In [34]:
scores=list(enumerate(cs[movie_id]))

In [35]:
sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
sorted_scores=sorted_scores[1:]

In [36]:
print(sorted_scores)

[(562, 0.6804138174397717), (187, 0.6123724356957946), (120, 0.5477225575051662), (125, 0.5477225575051662), (185, 0.5477225575051662), (5353, 0.5477225575051662), (51, 0.5000000000000001), (162, 0.5000000000000001), (294, 0.5000000000000001), (532, 0.5000000000000001), (2417, 0.5000000000000001), (6128, 0.5000000000000001), (13, 0.4714045207910318), (25, 0.4714045207910318), (39, 0.4714045207910318), (50, 0.4714045207910318), (147, 0.4714045207910318), (163, 0.4714045207910318), (539, 0.4714045207910318), (849, 0.4714045207910318), (1722, 0.4714045207910318), (10, 0.4629100498862757), (85, 0.4629100498862757), (102, 0.4629100498862757), (129, 0.4629100498862757), (139, 0.4629100498862757), (175, 0.4629100498862757), (186, 0.4629100498862757), (242, 0.4629100498862757), (489, 0.4629100498862757), (70, 0.4330127018922194), (72, 0.4330127018922194), (93, 0.4330127018922194), (5051, 0.4330127018922194), (5594, 0.4330127018922194), (7972, 0.4330127018922194), (8020, 0.4330127018922194), (1

# create a loop to print the first 7 similar movies

In [38]:
j=0
print('the 7 most recommended movies to' ,title, 'are:\n')
for item in sorted_scores:
    movie_title=df[df.movieId==item[0]]['title'].values
    print(j+1,movie_title)
    j=j+1
    if j>6:
        break

the 7 most recommended movies to Dead Man Walking (1995) are:

1 ['Welcome to the Dollhouse (1995)']
2 ['Party Girl (1995)']
3 []
4 ['Flirting With Disaster (1996)']
5 ['Net, The (1995)']
6 ['Butterflies Are Free (1972)']
7 []
